# Dogs Breed Classifier using CNN

## Importing Libraries

In [ ]:
import tensorflow as tf 
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

## Data Preprocessing

### Generating Images for Training Set

In [ ]:
trainDataGen = ImageDataGenerator(
    rescale = 1.0 / 255.0,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True 
)

trainingSet = trainDataGen.flow_from_directory(
    'data/training-set',
    target_size = (128, 128),
    batch_size = 32,
    class_mode = 'categorical'
)

### Writing breed classes in json file

In [ ]:
import json 

dog_classes_json = json.dumps(dog_classes)

with open('dog_classes.json', 'w+') as file:
    file.write(dog_classes_json)

### Generating images for Test set

In [ ]:
testDataGen = ImageDataGenerator(
    rescale = 1.0 / 255.0
)

testSet = testDataGen.flow_from_directory(
    'data/test-set',
    target_size = (128, 128),
    batch_size = 32,
    class_mode = 'categorical'
)

## Building The CNN

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout, Activation
from keras.models import Sequential

### Defining Neural Network Architecture

In [ ]:
model = Sequential()

### First Convolutional Layer

In [ ]:
model.add(Conv2D(32, (3, 3), input_shape=( 128, 128, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

### Second Convolution Layer

In [ ]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

### Third Convolutional Layer

In [ ]:
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

### Flattening

In [ ]:
model.add(Flatten()) 

### Full Connection

In [ ]:
model.add(Dense(64))
model.add(Activation('relu'))

### Output Layer

In [ ]:
model.add(Dense(26))
model.add(Activation('softmax'))

### Compiling the CNN

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Training Model on Train data

In [ ]:
model.fit(x = trainingSet, validation_data = testSet, epochs = 10)

### Saving Model to binary file

In [ ]:
model.save('model')